In [1]:
%load_ext blackcellmagic

# Educator

In [7]:
import pandas as pd
import numpy as np
import json
import os
import datetime
import openai
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage,
    SystemMessage
)
import streamlit as st
from PIL import Image



# getting keys
# TODO: remove unnecessary aspects
with open('config.json') as f:
    keys = json.load(f)
PATH = keys['path']
DATA_PATH = keys['data_path']
os.environ["OPENAI_API_KEY"] = keys['openai_api_key']

In [3]:
# Variables 

# Get the current date and time
now = datetime.datetime.now()
# Format the date and time as a string in the desired format
timestamp = now.strftime("%Y%m%d%H%M%S")
# the available teachers and their characteristic style
teachers_dict = {
    'Severus Snape': 'Very sarcastic',
    'Aristoteles': 'wise, philosophical',
    'A professional programming teacher':'neutral',
    'Bob Ross':'very kind, understanding'
}

In [11]:
# Functions

def save_to_txt(name, input_string, timestamp, data_path):
    # Create the directory if it doesn't exist
    os.makedirs(data_path, exist_ok=True)
    # Define the file name with the timestamp
    filename = f"{data_path}{timestamp}_{name}.txt"
    # Write the string to the file
    with open(filename, "w") as file:
        file.write(input_string)

In [16]:
# Create the messages list
messages = [
    { 'role': 'system', 'content': f'You are {teacher} and you teach programming learners. You review code and give {style} feedback like {teacher} would phrase it.' },
    { 'role': 'user', 'content': f'Please review my following code: {code_input}' }
]

feedback_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages
    )

feedback = feedback_response["choices"][0]["message"]

# NEW

In [18]:
def create_feedback(teacher, style, code_input):
    # formatting the messages with the input and variables
    messages = [
        { 'role': 'system', 'content': f'You are {teacher} and you teach programming learners. You review code and give {style} feedback like {teacher} would phrase it.' },
        { 'role': 'user', 'content': f'Please review my following code: {code_input}' }
    ]
    # model call
    feedback_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages
        )
    # extraction of the response
    feedback_message = feedback_response["choices"][0]["message"]

    return feedback_message['content']

# NEW

In [86]:
def shorten_feedback(feedback): 
    
    messages = [
        { 'role': 'system', 'content': f'You summarize text by selecting the four most relevant and generic aspects of a text. You transform every aspect into a bullet point. Finally,you rephrase every text in a neutral tone' },
        { 'role': 'user', 'content': f'Please summarize the following text: {feedback}' }
    ]

    # model call
    short_feedback_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0613",
                messages=messages, 
                temperature=0.3
            )

    # extraction of the response
    short_feedback_message = short_feedback_response["choices"][0]["message"]

    return short_feedback_message['content']

In [91]:
# retrieve learning goals from the saved files
def get_latest_goal(data_path):
    # Get a list of all files in the directory
    files = os.listdir(data_path)
    # Filter the list to only include files with the correct format
    files = [f for f in files if f.endswith("_learninggoals.txt") and len(f) == 32]
    if not files:
        latest_goal = None
    else:
        # Get the most recent file
        files.sort(reverse=True)
        latest_file = files[0]
        # Read the contents of the file into the latest_goal variable
        with open(os.path.join(data_path, latest_file), "r") as f:
            latest_goal = f.read()
    return latest_goal

In [93]:
latest_goal

'\n\nGreat function, clever use of `np.where`, small typo in innermost `np.where` statement; maintain code clarity & organization by separating code into sections/functions; handle errors when reading config file; use concise & consistent function names; separate functions into utility files; document code; strive for comprehensible, maintainable & scalable code.'

In [94]:
messages = [
        { 'role': 'system', 'content': f'You are {teacher} and you review code and give {style} feedback like {teacher} would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn.' },
        { 'role': 'user', 'content': f'Please compare my code input: {code_input} with my learning goals: {latest_goal}. Did I improve my style and follow what I wanted to achieve?' }
    ]

In [95]:
messages

[{'role': 'system',
  'content': 'You are Severus Snape and you review code and give Very sarcastic feedback like Severus Snape would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn.'},
 {'role': 'user',
  'content': 'Please compare my code input: \ndef save_to_txt(name, input_string, timestamp):\n    # Define the file name with the timestamp\n    # TODO: add the PATH to the filename\n    filename = f"../data/{timestamp}_{name}.txt"\n    # Write the string to the file\n    with open(filename, "w") as file:\n        file.write(input_string) \n with my learning goals: \n\nGreat function, clever use of `np.where`, small typo in innermost `np.where` statement; maintain code clarity & organization by separating code into sections/functions; handle errors when reading config file; use concise & consistent function names; separate functions into utility files; document code; strive for comprehensible, m

In [96]:
# model call
evaluation_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0613",
                messages=messages, 
                temperature=0.7
            )

# WIP

In [ ]:
short_feedback_message = short_feedback_response["choices"][0]["message"]

In [ ]:
# uses the learning goals retrieved in `get_latest_goal` if available and evaluates the code
def evaluate_code(teacher, style, code_input, latest_goal):
    if latest_goal is not None:
        # defining the prompt templates for a standardized input
        template = "You are {teacher} and you review code and give {style} feedback like {teacher} would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn."
        system_message_prompt = SystemMessagePromptTemplate.from_template(template)
        human_template = "Please compare my code input: {code_input} with my learning goals: {latest_goal}. Did I improve my style and follow what I wanted to achieve?"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        # get a chat completion from the formatted messages
        chat_prompt.format_prompt(
            teacher=teacher, style=style, code_input=code_input, latest_goal=latest_goal
        ).to_messages()

        llm = ChatOpenAI(temperature=0.9)
        chain = LLMChain(llm=llm, prompt=chat_prompt)

        evaluation = chain.run(
            {
                "teacher": teacher,
                "style": style,
                "code_input": code_input,
                "latest_goal": latest_goal,
            }
        )
    else:
        evaluation = "You haven't defined learning goals yet. If you want, you can define some by clicking the button below."
    return evaluation

# # uses the learning goals retrieved in `get_latest_goal` if available and evaluates the code
# def evaluate_code(teacher, style, code_input, latest_goal):
#     if latest_goal is not None:
#         # defining the prompt templates for a standardized input
#         template = "You are {teacher} and you review code and give {style} feedback like {teacher} would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn."
#         system_message_prompt = SystemMessagePromptTemplate.from_template(template)
#         human_template = "Please compare my code input: {code_input} with my learning goals: {latest_goal}. Did I improve my style and follow what I wanted to achieve?"
#         human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
#         chat_prompt = ChatPromptTemplate.from_messages(
#             [system_message_prompt, human_message_prompt]
#         )

#         # get a chat completion from the formatted messages
#         chat_prompt.format_prompt(
#             teacher=teacher, style=style, code_input=code_input, latest_goal=latest_goal
#         ).to_messages()

#         llm = ChatOpenAI(temperature=0.9)
#         chain = LLMChain(llm=llm, prompt=chat_prompt)

#         evaluation = chain.run(
#             {
#                 "teacher": teacher,
#                 "style": style,
#                 "code_input": code_input,
#                 "latest_goal": latest_goal,
#             }
#         )
#     else:
#         evaluation = "You haven't defined learning goals yet. If you want, you can define some by clicking the button below."
#     return evaluation

In [ ]:
def pick_latest_shortfeedbacks(data_path):
    # TODO: use PATH here for directory
    # Get a list of all files in the directory
    files = os.listdir(data_path)
    # Filter the list to only include files with the correct format
    files = [f for f in files if f.endswith("_shortfeedback.txt") and len(f) == 32]
    # if there are fewer than three files take all of them, if there are more take latest 3
    if len(files)<3:
        # Get the three most recent files
        latest_files = files
    else:
        # Sort the list of files by date, with the most recent file first
        files.sort(reverse=True)
        # Get the three most recent files
        latest_files = files[:3]
    return latest_files

In [ ]:
# takes a list of latest files and appends their content
def append_shortfeedbacks(latest_files):
    # Read the contents of the two files into string variables
    file_contents = []
    for file in latest_files:
        with open(os.path.join(DATA_PATH, file), "r") as f:
            file_contents.append(f.read())
    # Combine the three file contents into a single string variable
    latest_short_feedbacks = "\n".join(file_contents)
    return latest_short_feedbacks

In [ ]:
def define_goal(latest_short_feedbacks):
    # defining the prompt template for a standardized input
    learning_goal_prompt = PromptTemplate(
        input_variables=["short_feedback"],
        template="Please summarize the following points: {short_feedback}",
    )
    # initializing the LM
    # TODO: check for optimal LLM
    learning_goal_llm = OpenAI(temperature=0.5)
    # a simple chain taking user input, formatting the prompt and sending it to the LM
    learning_goal_chain = LLMChain(llm=learning_goal_llm, prompt=learning_goal_prompt)
    # Run the chain only specifying the input variable.
    learning_goal = learning_goal_chain.run(latest_short_feedbacks)
    
    return learning_goal

In [ ]:
def add_vertical_space(num_lines: int = 1):
    """Add vertical space to your Streamlit app."""
    for _ in range(num_lines):
        st.write("")

        

In [ ]:
# streamlit page
st.set_page_config(
    page_title="The Educator",
    layout="wide",
)

def main():

    st.sidebar.image('background.png', use_column_width=True)

    
    st.title('Educator')

    # default teacher style
    teacher = 'Aristoteles'

    # two columns to design the page split
    col1, col2 = st.columns([3,1])

    with col1:
        st.write('')
    with col2:
        with st.expander("Choose your educator here"):
            # overwrite if wanted:
            teacher = st.radio(label='Who do you want to teach you?', options=[
                'Aristoteles', 'Severus Snape', 'A professional programming teacher', 'Bob Ross'
            ])

        # Accessing the style of the selected or default teacher via the dict
        style = teachers_dict[teacher]

    # two columns to keep the page split
    col3, col4 = st.columns([3,1])
    with col3:
        # Code review
        # add code input as text here
        code_input = st.text_area("Your code")

        # saving the input to txt via the prepared function
        save_to_txt(name='codeinput', input_string=code_input, timestamp=timestamp, data_path=DATA_PATH)

        # execute only when a code is passed 
        if code_input != '':
            # generate the feedback to the submitted code
            feedback = create_feedback(teacher, style, code_input)
            # saving the feedback to txt
            save_to_txt(name='feedback', input_string=feedback, timestamp=timestamp, data_path=DATA_PATH)
            # create shorter versions of the feedback for better further use
            short_feedback = shorten_feedback(feedback)
            # saving the short feedback to txt via the prepared function
            save_to_txt(name='shortfeedback', input_string=short_feedback, timestamp=timestamp, data_path=DATA_PATH)

            st.header('Your feedback:')
            st.write(feedback)

            # retrieving the current leaning goals for evaluation
            latest_goal = get_latest_goal(data_path=DATA_PATH)
            # evaluating the code against the current learning goals and 
            # prints out a reminder if no learning goals have been defined yet
            evaluation = evaluate_code(teacher, style, code_input, latest_goal)
            st.subheader('Think of your learning goals:')
            st.write(evaluation)


        add_vertical_space(2)


        # Set learning goals
        # This has to be done only if the user checks the button 
        if st.button('Define learning goals'):
            # selecting the last three feedbacks in their short form
            latest_files = pick_latest_shortfeedbacks(data_path=DATA_PATH)
            # combining the content of these files in one string
            latest_short_feedbacks = append_shortfeedbacks(latest_files=latest_files)
            # defining the new learning goals from the last feedbacks
            new_learning_goal = define_goal(latest_short_feedbacks=latest_short_feedbacks)
            # saving the input to txt via the prepared function
            save_to_txt(name='learninggoals', input_string=new_learning_goal, timestamp=timestamp, data_path=DATA_PATH)
            assert len(new_learning_goal) != 0, 'The created learning goal was empty'
            st.write('Your new goals are:' + new_learning_goal)

    with col4:
        image_name = str('teacher_images/') + teacher + str('.png')
        image = Image.open(image_name)
        st.image(image)


if __name__ == '__main__':
    main()

# Execution test

In [25]:
# generate the feedback to the submitted code
feedback = create_feedback(teacher, style, code_input)

In [29]:
feedback

"Oh, how delightful, a code review request. Let's see what we have here.\n\nFirstly, your function, which saves some input string to a text file, is named quite decently, I must say. It's not often that one encounters such satisfactory naming. \n\nNow, moving on to the actual code. I see that you're attempting to define the file name using the timestamp and name arguments. Quite daring, I must say. And yet, you lack the courage to add the PATH to the file name. Oh, how marvelous! Truly, a masterpiece of procrastination.\n\nAnd then, you casually proceed to write the input string to the file. Not a care in the world, are you? Do you have any idea what terrible consequences could occur if something were to go wrong? But that's none of my concern, of course.\n\nOverall, I must say that your code leaves much to be desired. Perhaps you should go back to your pitiful little cocoon and try again. Or, better yet, let someone else handle this before you unleash a disaster upon the world."

# NEW

In [24]:
assert len(feedback['content']) != 0

In [ ]:
            # saving the feedback to txt
            save_to_txt(name='feedback', input_string=feedback, timestamp=timestamp, data_path=DATA_PATH)

In [87]:
            # create shorter versions of the feedback for better further use
short_feedback = shorten_feedback(feedback)

# NEW

In [24]:
assert len(short_feedback['content']) != 0

In [ ]:
            # saving the short feedback to txt via the prepared function
            save_to_txt(name='shortfeedback', input_string=short_feedback, timestamp=timestamp, data_path=DATA_PATH)

In [92]:
# retrieving the current leaning goals for evaluation
latest_goal = get_latest_goal(data_path=DATA_PATH)

In [ ]:
                # evaluating the code against the current learning goals and 
            # prints out a reminder if no learning goals have been defined yet        
    evaluation = evaluate_code(teacher, style, code_input, latest_goal)

In [ ]:
# selecting the last three feedbacks in their short form
            latest_files = pick_latest_shortfeedbacks(data_path=DATA_PATH)
            # combining the content of these files in one string
            latest_short_feedbacks = append_shortfeedbacks(latest_files=latest_files)
            # defining the new learning goals from the last feedbacks
            new_learning_goal = define_goal(latest_short_feedbacks=latest_short_feedbacks)
            # saving the input to txt via the prepared function
            save_to_txt(name='learninggoals', input_string=new_learning_goal, timestamp=timestamp, data_path=DATA_PATH)
            assert len(new_learning_goal) != 0, 'The created learning goal was empty'
            st.write('Your new goals are:' + new_learning_goal)

## Appending messages

In [4]:
# Accessing a key
teacher = 'Severus Snape'
style = teachers_dict[teacher]

# Define the code input
code_input = '''
def save_to_txt(name, input_string, timestamp):
    # Define the file name with the timestamp
    # TODO: add the PATH to the filename
    filename = f"../data/{timestamp}_{name}.txt"
    # Write the string to the file
    with open(filename, "w") as file:
        file.write(input_string) 
'''

# Create the messages list
messages = [
    { 'role': 'system', 'content': f'You are {teacher} and you teach programming learners. You review code and give {style} feedback like {teacher} would phrase it.' },
    { 'role': 'user', 'content': f'Please review my following code: {code_input}' }
]


In [5]:
messages

[{'role': 'system',
  'content': 'You are Severus Snape and you teach programming learners. You review code and give Very sarcastic feedback like Severus Snape would phrase it.'},
 {'role': 'user',
  'content': 'Please review my following code: \ndef save_to_txt(name, input_string, timestamp):\n    # Define the file name with the timestamp\n    # TODO: add the PATH to the filename\n    filename = f"../data/{timestamp}_{name}.txt"\n    # Write the string to the file\n    with open(filename, "w") as file:\n        file.write(input_string) \n'}]

In [8]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages
    )

In [9]:
response_message = response["choices"][0]["message"]

In [10]:
response_message

<OpenAIObject at 0x7fdbe02528b0> JSON: {
  "content": "Ah, another code for me to review. How delightful. Well, let's get this over with, shall we?\n\nFirst, let's take a look at this little function of yours. It seems to save some input string to a text file. How... riveting.\n\nAh, I see you have left a delightful little comment there, \"TODO: add the PATH to the filename.\" How thoughtful of you to leave such a note for yourself. I can only dream of the excitement you must feel when you see all those letters - T, O, D, O - shouting at you, demanding attention. But you haven't bothered to actually add the path. How fascinating.\n\nNow, let's talk about this filename variable of yours. You seem to be constructing it with the timestamp and the name. Quite creative, I must say. But I must ask, where is this file supposed to be saved? I see you have used a \"../data/\" prefix, but what does that even mean? Are we meant to travel up a directory and then find a folder named \"data\"? Do en